In [ ]:
from __future__ import print_function
import os,sys
import chart_studio.plotly as py
import plotly.graph_objects as go
import numpy as np
from readparquet import UBParquetReader
from detectoroutline import DetectorOutline

In [ ]:
import lardly
detdata = lardly.DetectorOutline()

In [ ]:
# load utility to draw TPC outline
detdata = DetectorOutline()

# define some colors for particle types
ssnet_color_array = np.array( ((0,0,0),      # bg
                               (255,0,0),    # electron
                               (0,255,0),    # gamma
                               (0,0,255),    # muon
                               (255,0,255),  # pion
                               (0,255,255),  # proton
                               (0,0,0)),     # other
                               dtype=np.float ) 

In [ ]:
# DATA FILES
datafile="temp.parquet"

reader = UBParquetReader(datafile)
NENTRIES = reader.nentries

In [ ]:
# Get entry data
ENTRY = 0
data = reader.get_entry(ENTRY)

In [ ]:
# GENERATE PLOTLY OBJECTS FOR VISUALIZATION
plotdata = []
nvoxels = data["voxcoord"].shape[0]
# We need to retrieved the 3d positions
pos3d = data["voxcoord"].astype(np.float)*1.0
pos3d[:,1] -= 117.0 


In [ ]:
# Plot Particle ID (semenatic segmentation network, "SSNet") labels
no_ghost_points = True

ssnetlabels = data["voxssnet"]
print("ssnetlabels: ",ssnetlabels.shape)
ssnetcol = np.zeros((nvoxels,3))
for i in range(6):
   ssnetcol[ ssnetlabels==i ] = ssnet_color_array[i,:]

if no_ghost_points:
    pos = pos3d[data['voxlabel']==1]
    ssnetcol = ssnetcol[data['voxlabel']==1]
    print("removed ghost voxels: ",pos3d.shape)
else:
    pos = pos3d


ssnetplot = {
    "type":"scatter3d",
    "x":pos[:,0],
    "y":pos[:,1],
    "z":pos[:,2],
        "mode":"markers",
        "name":"ssnet",
        "marker":{"color":ssnetcol,"size":1}
    }

detlines = detdata.getlines(color=(10,10,10))

# DATA
ssnet_plot_data = [ssnetplot] + detdata.getlines(color=(10,10,10))

# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='SSNET LABELS',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": 1, "y": 1, "z": 1},
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=ssnet_plot_data, layout=layout)
fig.show()